
#  
## Calibrate-Emulate-Sample Demo: Application to a Cloud Microphysics Toy Model

This notebook demonstrates how parameters of cloud droplet size distributions can be learned from data generated by Cloudy, a cloud microphysics toy model that can be downloaded here: https://github.com/climate-machine/Cloudy.jl. 

Cloudy takes the following input:
* parameters defining a **cloud droplet size distribution** (e.g., parameters defining a gamma distribution or an exponential distribution)
* a **''kernel''** defining the efficiency with which cloud droplets collide and coalesce (i.e., stick together and form bigger droplets). 

Cloudy then simulates how the cloud droplet size distribution evolves over time as a result of the droplet interactions defined by the given kernel. It does so by computing a (user-defined) number of moments of the distribution and propagating them forward in time. The simulation stops after a user-defined amount of time $T$. 
Treating the kernel as given, Cloudy can be viewed as a function $G(u): \mathbb{R}^p \rightarrow \mathbb{R}^d$ that maps the vector of parameters describing the initial droplet distribution, $u = [u_1, u_2, \ldots, u_p]$, to a vector of moments, $M = [M_0^T, M_1^T, \ldots, M_d^T]$

The model equation assumes that we have observations $y$ of the moments coming from some observing system, and that these observations are corrupted by additive noise $\eta$ such that
\begin{equation}
    y = G(u) + \eta,
\end{equation}

where the noise $\eta$ is drawn from a d-dimensional Gaussian with distribution $N(0, \Gamma_y)$.

In this demo, we test the calibrate-emulate-sample framework in a **''perfect-model'' setting**, which means that the observations $y$ do not come from some external observing system but are generated by running Cloudy with the  parameters set to their ''true'' values. 

**Given knowledge of the artificial observations $y$, the forward model $G: \mathbb{R}^p \rightarrow \mathbb{R}^d$, and some information about the noise level such as its size or distribution (but not its value), the inverse problem we want to solve is to find the unknown parameters $u$.**

A comprehensive treatment of the calibrate-emulate-sample approach to Bayesian inverse problems can be found in Cleary et al., 2020: https://arxiv.org/pdf/2001.03689.pdf

In a one-sentence summary, the **calibrate** step of the algorithm consists of an Ensemble Kalman Inversion that is used to find good training points for a Gaussian process regression, which in turn is used as a replacement (**emulator**) of the original forward model $G$ in the subsequent Markov chain Monte Carlo **sampling** of the posterior distributions of the unknown parameters.

**Final remarks**: Calibrate-emulate-sample was developed to solve inverse problems in situations when running the forward model $G$ is computationally expensive. Cloudy is very cheap to run and it would be computationally feasible to skip the ''calibrate'' and ''emulate'' steps, but this being a demo, we will walk the user through the full pipeline. Applying the full pipeline may also improve the result even when it's not necessary for computational reasons, e.g., because the Gaussian process regression smooths the cost function.

<img src="cloudy_model.jpg" alt="Cloudy" style="width:600px">



##### Load modules

In [ ]:
# Import Cloudy modules
using Cloudy.ParticleDistributions
using Cloudy.KernelTensors
using Cloudy.Sources

In [ ]:
# Import modules
using Distributions  # probability distributions and associated functions 
using StatsBase
using LinearAlgebra
using StatsPlots
using GaussianProcesses

In [ ]:
# Import Calibrate-Emulate-Sample modules (adjust paths 
# if necessary)
include("CalibrateEmulateSample.jl/src/EKI.jl")
include("CalibrateEmulateSample.jl/src/Observations.jl")
include("CalibrateEmulateSample.jl/src/GPEmulator.jl")
include("CalibrateEmulateSample.jl/src/MCMC.jl")
include("CalibrateEmulateSample.jl/src/GModel.jl")
include("CalibrateEmulateSample.jl/src/Utilities.jl")

### Set up experiment

In [ ]:
# Define the parameters that we want to learn
# We assume that the true particle mass distribution is a 
# Gamma distribution with parameters N0_true, θ_true, k_true
param_names = ["N0", "θ", "k"]
n_param = length(param_names)

# Define the data from which we want to learn these parameters
data_names = ["M0", "M1", "M2"]
moments = [0.0, 1.0, 2.0]
n_moments = length(moments)

### Define priors

In [ ]:
# Assume lognormal priors for all three parameters
# Note: For the model G (=Cloudy) to run, N0 needs to be 
# nonnegative, and θ and k need to be positive. 
# The EKI update can result in violations of these constraints - 
# therefore, we perform CES in log space, i.e., we try to find 
# the logarithms of the true parameters (and of course, the actual
# parameters could then simply be obtained by exponentiating the 
# final results). 

function logmean_and_logstd(μ, σ)
    σ_log = sqrt(log(1.0 + σ^2/μ^2))
    μ_log = log(μ / (sqrt(1.0 + σ^2/μ^2)))
    return μ_log, σ_log
end

N0_true = 300.0 
θ_true = 1.5597  
k_true = 0.0817                  
params_true = [N0_true, θ_true, k_true]
# Note that dist_true has to be a Cloudy distribution, not a 
# "regular" Julia distribution
dist_true = ParticleDistributions.Gamma(N0_true, θ_true, k_true)

logmean_N0, logstd_N0 = logmean_and_logstd(280., 40.)
logmean_θ, logstd_θ = logmean_and_logstd(3.0, 1.5)
logmean_k, logstd_k = logmean_and_logstd(0.5, 0.5)

priors = [Distributions.Normal(logmean_N0, logstd_N0),  # prior on N0
          Distributions.Normal(logmean_θ, logstd_θ),    # prior on θ
          Distributions.Normal(logmean_k, logstd_k)]    # prior on k 

### Cloudy settings

In [ ]:
# Collision-coalescence kernel to be used in Cloudy
coalescence_coeff = 1/3.14/4
kernel_func = x -> coalescence_coeff
kernel = CoalescenceTensor(kernel_func, 0, 100.0)

# Time period over which to run Cloudy
tspan = (0., 0.5)  

### Generate truth

In [ ]:
# Generate (artificial) truth samples
g_settings_true = GModel.GSettings(kernel, dist_true, moments, tspan)
yt = GModel.run_G(params_true, g_settings_true, 
                  ParticleDistributions.update_params, 
                  ParticleDistributions.moment,
                  Sources.get_int_coalescence)
n_samples = 100
samples = zeros(n_samples, length(yt))
noise_level = 0.05
Γy = noise_level^2 * convert(Array, Diagonal(yt))
μ = zeros(length(yt))

# Add noise
for i in 1:n_samples
    samples[i, :] = yt + noise_level^2 * rand(MvNormal(μ, Γy))
end

truth = Observations.Obs(samples, Γy, data_names)

# Calibrate: Ensemble Kalman Inversion


In [ ]:
log_transform(a::AbstractArray) = log.(a)
exp_transform(a::AbstractArray) = exp.(a)

In [ ]:
N_ens = 50 # number of ensemble members
N_iter = 5 # number of EKI iterations
# initial parameters: N_ens x N_params
initial_params = EKI.construct_initial_ensemble(N_ens, priors; rng_seed=6)
ekiobj = EKI.EKIObj(initial_params, 
                    param_names, truth.mean, truth.cov)

In [ ]:
# Initialize a ParticleDistribution with dummy parameters
# The parameters will then be set in run_cloudy_ensemble
dummy = 1.0
dist_type = ParticleDistributions.Gamma(dummy, dummy, dummy)
g_settings = GModel.GSettings(kernel, dist_type, moments, tspan)

In [ ]:
# EKI iterations
for i in 1:N_iter
    # Note that the parameters are exp-transformed for use as input
    # to Clouedy
    params_i = deepcopy(exp_transform(ekiobj.u[end]))
    g_ens = GModel.run_G_ensemble(params_i, 
                                  g_settings,
                                  ParticleDistributions.update_params,
                                  ParticleDistributions.moment,
                                  Sources.get_int_coalescence)
    EKI.update_ensemble!(ekiobj, g_ens) 
end

In [ ]:
# EKI results: Has the ensemble collapsed toward the truth?
println("True parameters: ")
println(params_true)

println("\nEKI results:")
println(mean(deepcopy(exp_transform(ekiobj.u[end])), dims=1))

# Emulate: Gaussian Process Regression

In [ ]:
gppackage = GPEmulator.GPJL()
pred_type = GPEmulator.YType()
# Construct kernel:
# Sum kernel consisting of Matern 5/2 ARD kernel, a Squared
# Exponential Iso kernel and white noise
# Note that the kernels take the signal standard deviations on a 
# log scale as input.
len1 = 1.0
kern1 = SE(len1, 1.0)
len2 = zeros(3)
kern2 = Mat52Ard(len2, 0.0)
# regularize with white noise
white = Noise(log(2.0))
# construct kernel
GPkernel =  kern1 + kern2 + white
    
u_tp, g_tp = Utilities.extract_GP_tp(ekiobj, N_iter)
normalized = true
gpobj = GPEmulator.GPObj(u_tp, g_tp, gppackage; GPkernel=GPkernel, 
                         normalized=normalized, prediction_type=pred_type)

In [ ]:
# Check how well the Gaussian Process regression predicts on the
# true parameters
y_mean, y_var = GPEmulator.predict(gpobj, reshape(log.(params_true), 1, :))

println("GP prediction on true parameters: ")
println(vec(y_mean))
println("true data: ")
println(truth.mean)

# Sample: Markov chain Monte Carlo

In [ ]:
# initial values
u0 = vec(mean(u_tp, dims=1))
println("initial parameters: ", u0)

# MCMC parameters    
mcmc_alg = "rwm" # random walk Metropolis

# First let's run a short chain to determine a good step size
burnin = 0
step = 0.1 # first guess
max_iter = 5000
yt_sample = truth.mean
mcmc_test = MCMC.MCMCObj(yt_sample, truth.cov, 
                         priors, step, u0, 
                         max_iter, mcmc_alg, burnin)
new_step = MCMC.find_mcmc_step!(mcmc_test, gpobj)

In [ ]:
# Now begin the actual MCMC
println("Begin MCMC - with step size ", new_step)
u0 = vec(mean(u_tp, dims=1))

# reset parameters 
burnin = 1000
max_iter = 500000

mcmc = MCMC.MCMCObj(yt_sample, truth.cov, priors, 
                    new_step, u0, max_iter, mcmc_alg, burnin)
MCMC.sample_posterior!(mcmc, gpobj, max_iter)

In [ ]:
posterior = MCMC.get_posterior(mcmc)      

post_mean = mean(posterior, dims=1)
post_cov = cov(posterior, dims=1)
println("post_mean")
println(post_mean)
println("post_cov")
println(post_cov)
println("D util")
println(det(inv(post_cov)))
println(" ")

In [ ]:
# Plot the posteriors together with the priors and the true 
# parameter values
using StatsPlots; 

true_values = [log(N0_true) log(θ_true) log(k_true)]
n_params = length(true_values)

for idx in 1:n_params
    if idx == 1
        param = "N0"
        xs = collect(4.5:0.01:6.5)
    elseif idx == 2
        param = "Theta"
        xs = collect(-1.0:0.01:2.5)
    elseif idx == 3
        param = "k"
        xs = collect(-4.0:0.01:1.0)
    else
        throw("not implemented")
    end

    label = "true " * param
    histogram(posterior[:, idx], bins=100, normed=true, 
              fill=:slategray, lab="posterior")
    plot!(xs, mcmc.prior[idx], w=2.6, color=:blue, lab="prior")
    plot!([true_values[idx]], seriestype="vline", w=2.6, lab=label)

    title!(param)
    StatsPlots.savefig("/home/melanie/Desktop/posterior_"*param*".png")
end